In [4]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import os
from scipy.stats import ttest_1samp
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

In [10]:
df = pd.read_csv('combined_df.csv')


In [14]:
def kmeans(df):
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import Normalizer
    from sklearn.preprocessing import normalize

    df = df.dropna()
    X = df[['Rating', 'Reviews', 'Size', 'Installs', 'Price']]
    X = X.apply(lambda x: x.fillna(x.mean()),axis=0)
    X = X.values
    X = normalize(X)
    kmeans = KMeans(n_clusters=10)
    kmeans.fit(X)
    y_kmeans = kmeans.predict(X)
    df['Cluster'] = y_kmeans
    return df


In [15]:
kmeans(df)

C:\Users\new\AppData\Local\Temp\ipykernel_22480\2304319267.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cluster'] = y_kmeans


,App,Type,Genres,App Id,Rating Count,Currency,Developer Id,Developer Website,Developer Email,Released,...,Processed_Genres,Stemmed_Genres,Lemmatized_Genres,Processed_Developer Address,Stemmed_Developer Address,Lemmatized_Developer Address,Processed_Category,Stemmed_Category,Lemmatized_Category,Cluster
0,Smoke Effect Photo Maker - Smoke Editor,0,Art & Design,com.devanzu.smokeeffectphotomaker.smokeeditor,1114.0,USD,Anzu+Dev,https://devanzu.blogspot.com/,devanzu9@gmail.com,26-Apr-18,...,"['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...","['1', '6', '0', '0', ' ', 'p', 'h', 'h', 'e', ...","['1', '6', '0', '0', ' ', 'a', 'm', 'p', 'h', ...","['1', '6', '0', '0', ' ', 'A', 'm', 'p', 'h', ...","['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...",0
1,Infinite Painter,0,Art & Design,com.brakefield.painter,92790.0,USD,8.16109e+18,https://www.infinitestudio.art/,support@infinitestudio.art,2-Apr-12,...,"['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...","['1', '1', '5', '5', ' ', 'E', 'N', 'L', 'E', ...","['1', '1', '5', '5', ' ', 's', 'e', 'm', 'i', ...","['1', '1', '5', '5', ' ', 'S', 'E', 'M', 'I', ...","['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...",0
2,Garden Coloring Book,0,Art & Design,com.eyewind.colorfit.garden,16599.0,USD,Colorfit,https://www.ilyon.net/,colorfit.app@outlook.com,1-Apr-16,...,"['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...","['F', 'l', ' ', '1', '1', ' ', ' ', '1', '/', ...","['f', 'l', 'a', 't', ' ', '1', '1', ' ', ' ', ...","['F', 'l', 'a', 't', ' ', '1', '1', ' ', ' ', ...","['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...",0
3,Tattoo Name On My Photo Editor,0,Art & Design,com.Voodamdee.Photos.Camera.TattooNameOnMyPhot...,110.0,USD,Voodamdee,http://voodamdee.tk/website/,odownrang36@gmail.com,19-Oct-18,...,"['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...","['1', '6', '0', '0', ' ', 'p', 'h', 'h', 'e', ...","['1', '6', '0', '0', ' ', 'a', 'm', 'p', 'h', ...","['1', '6', '0', '0', ' ', 'A', 'm', 'p', 'h', ...","['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...",1
4,Tattoo Name On My Photo Editor,0,Art & Design,com.fashion.tattoo.name.my.photo.editor,50147.0,USD,iDroid+Solution,https://www.ilyon.net/,trendingmobileapps@gmail.com,10-Mar-17,...,"['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...","['u', 'r', ',', ' ', 'G', 'u', 'j', 'r', ',', ...","['s', 'u', 'r', 'a', 't', ',', ' ', 'g', 'u', ...","['S', 'u', 'r', 'a', 't', ',', ' ', 'G', 'u', ...","['r', ' ', '&', ' ', 'e', 'g', 'n']","['a', 'r', 't', ' ', '&', ' ', 'd', 'e', 's', ...","['A', 'r', 't', ' ', '&', ' ', 'D', 'e', 's', ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,Poker Pro.Fr,0,Card,air.com.coalaa.itexasfr,5780.0,USD,8.15217E+18,http://itexas.coalaa.com,samyip@boyaa.com,25-Apr-13,...,"['C', 'r']","['c', 'a', 'r', 'd']","['C', 'a', 'r', 'd']","['U', 'g', 'l', 'n', ' ', 'H', 'u', 'e', ',', ...","['u', 'g', 'l', 'a', 'n', 'd', ' ', 'h', 'o', ...","['U', 'g', 'l', 'a', 'n', 'd', ' ', 'H', 'o', ...","['C', 'r']","['c', 'a', 'r', 'd']","['C', 'a', 'r', 'd']",7
6246,Golden Dictionary (FR-AR),0,Books & Reference,com.goldensoft.dictionaryfrfree,7645.0,USD,6.58252E+18,http://www.golden-soft.net,info@golde